Today is friday

## Import Data

In [1]:
# install yfinance
!pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 514 kB/s 
     |████████████████████████████████| 6.4 MB 11.5 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.28.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [2]:
import yfinance as yf
import pandas as pd
import numpy as np

Volume : volume measures the number of a stock's shares that are traded on a stock exchange in a day or a period of time.

In [3]:
ticker = '^GSPC'
tick = yf.Ticker(ticker)
dataset = tick.history(start='2020-01-01', end='2021-12-31', rounding=True).iloc[:,:-2]
dataset.head()

,Open,High,Low,Close,Volume
Date,,,,,
2020-01-02,3244.67,3258.14,3235.53,3257.85,3458250000
2020-01-03,3226.36,3246.15,3222.34,3234.85,3461290000
2020-01-06,3217.55,3246.84,3214.64,3246.28,3674070000
2020-01-07,3241.86,3244.91,3232.43,3237.18,3420380000
2020-01-08,3238.59,3267.07,3236.67,3253.05,3720890000


Volume on Yahoo Finance's charts are the physical number of shares traded of that stock (not dollar amount) for your given period of time. This is typically what volume is reported as, so it would be safe to assume their endpoint follows the same protocol.
https://stackoverflow.com/questions/38545299/volume-yahoo-finance

In [4]:
dataset.shape

(504, 5)

## Indicators

### Simple Moving Average

In [5]:
# moving average
dataset['MA_10'] = dataset.Close.rolling(10).mean()
dataset['MA_30'] = dataset.Close.rolling(30).mean()
dataset['MA_50'] = dataset.Close.rolling(50).mean()
dataset['MA_70'] = dataset.Close.rolling(70).mean()
dataset['MA_90'] = dataset.Close.rolling(90).mean()
dataset.head(5)

,Open,High,Low,Close,Volume,MA_10,MA_30,MA_50,MA_70,MA_90
Date,,,,,,,,,,
2020-01-02,3244.67,3258.14,3235.53,3257.85,3458250000,NaN,NaN,NaN,NaN,NaN
2020-01-03,3226.36,3246.15,3222.34,3234.85,3461290000,NaN,NaN,NaN,NaN,NaN
2020-01-06,3217.55,3246.84,3214.64,3246.28,3674070000,NaN,NaN,NaN,NaN,NaN
2020-01-07,3241.86,3244.91,3232.43,3237.18,3420380000,NaN,NaN,NaN,NaN,NaN
2020-01-08,3238.59,3267.07,3236.67,3253.05,3720890000,NaN,NaN,NaN,NaN,NaN


In [6]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
M=40
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x=dataset.index[-M:], y=dataset.Close[-M:], name='close'), secondary_y=False)
fig.add_trace(go.Scatter(x=dataset.index[-M:], y=dataset.MA_10[-M:], name='MA_10'), secondary_y=False)
fig.add_trace(go.Scatter(x=dataset.index[-M:], y=dataset.MA_50[-M:], name='MA_50'), secondary_y=False)
fig.add_trace(go.Scatter(x=dataset.index[-M:], y=dataset.MA_90[-M:], name='MA_90'), secondary_y=False)


In [7]:
dataset2 = dataset.copy()
for i in range(5,200,5):
  dataset2['MA_'+str(i)] = dataset2.Close.rolling(i).mean()
dataset2.head()

,Open,High,Low,Close,Volume,MA_10,MA_30,MA_50,MA_70,MA_90,...,MA_150,MA_155,MA_160,MA_165,MA_170,MA_175,MA_180,MA_185,MA_190,MA_195
Date,,,,,,,,,,,,,,,,,,,,,
2020-01-02,3244.67,3258.14,3235.53,3257.85,3458250000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-03,3226.36,3246.15,3222.34,3234.85,3461290000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-06,3217.55,3246.84,3214.64,3246.28,3674070000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-07,3241.86,3244.91,3232.43,3237.18,3420380000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-08,3238.59,3267.07,3236.67,3253.05,3720890000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Exponential Weighted Moving Average


In [8]:
dataset['EWM_10'] = dataset.Close.ewm(span=10, adjust=False).mean()
dataset['EWM_30'] = dataset.Close.ewm(span=10, adjust=False).mean()
dataset['EWM_50'] = dataset.Close.ewm(span=50, adjust=False).mean()
dataset['EWM_70'] = dataset.Close.ewm(span=10, adjust=False).mean()
dataset['EWM_90'] = dataset.Close.ewm(span=100, adjust=False).mean()

dataset.head().round(2)

,Open,High,Low,Close,Volume,MA_10,MA_30,MA_50,MA_70,MA_90,EWM_10,EWM_30,EWM_50,EWM_70,EWM_90
Date,,,,,,,,,,,,,,,
2020-01-02,3244.67,3258.14,3235.53,3257.85,3458250000,NaN,NaN,NaN,NaN,NaN,3257.85,3257.85,3257.85,3257.85,3257.85
2020-01-03,3226.36,3246.15,3222.34,3234.85,3461290000,NaN,NaN,NaN,NaN,NaN,3253.67,3253.67,3256.95,3253.67,3257.39
2020-01-06,3217.55,3246.84,3214.64,3246.28,3674070000,NaN,NaN,NaN,NaN,NaN,3252.32,3252.32,3256.53,3252.32,3257.17
2020-01-07,3241.86,3244.91,3232.43,3237.18,3420380000,NaN,NaN,NaN,NaN,NaN,3249.57,3249.57,3255.77,3249.57,3256.78
2020-01-08,3238.59,3267.07,3236.67,3253.05,3720890000,NaN,NaN,NaN,NaN,NaN,3250.20,3250.20,3255.66,3250.20,3256.70


### Relative Strength Index (RSI)


In [9]:
def rsi(df, column, N = 14):
    delta   = df[column].diff()
    
    up_days = delta.copy()
    up_days[delta <= 0] = 0.0
    
    down_days = delta.copy()
    down_days[delta > 0] = 0.0
    down_days = abs(down_days)
    
    RS_up = up_days.rolling(N).mean()
    RS_down = down_days.rolling(N).mean()
    
    rsi = 100 - 100 / (1 + RS_up / RS_down)
    
    return rsi.values

In [10]:
dataset['RSI_14'] = rsi(dataset, 'Close', N = 14)
dataset['RSI_24'] = rsi(dataset, 'Close', N = 24)
dataset['RSI_34'] = rsi(dataset, 'Close', N = 34)
dataset['RSI_44'] = rsi(dataset, 'Close', N = 44)
dataset['RSI_54'] = rsi(dataset, 'Close', N = 44)
dataset['RSI_64'] = rsi(dataset, 'Close', N = 44)
dataset.head()

,Open,High,Low,Close,Volume,MA_10,MA_30,MA_50,MA_70,MA_90,...,EWM_30,EWM_50,EWM_70,EWM_90,RSI_14,RSI_24,RSI_34,RSI_44,RSI_54,RSI_64
Date,,,,,,,,,,,,,,,,,,,,,
2020-01-02,3244.67,3258.14,3235.53,3257.85,3458250000,NaN,NaN,NaN,NaN,NaN,...,3257.850000,3257.850000,3257.850000,3257.850000,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-03,3226.36,3246.15,3222.34,3234.85,3461290000,NaN,NaN,NaN,NaN,NaN,...,3253.668182,3256.948039,3253.668182,3257.394554,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-06,3217.55,3246.84,3214.64,3246.28,3674070000,NaN,NaN,NaN,NaN,NaN,...,3252.324876,3256.529685,3252.324876,3257.174464,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-07,3241.86,3244.91,3232.43,3237.18,3420380000,NaN,NaN,NaN,NaN,NaN,...,3249.571262,3255.770874,3249.571262,3256.778534,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-08,3238.59,3267.07,3236.67,3253.05,3720890000,NaN,NaN,NaN,NaN,NaN,...,3250.203760,3255.664173,3250.203760,3256.704702,NaN,NaN,NaN,NaN,NaN,NaN


## Return

In [11]:
import numpy as np

In [12]:
dataset['Prct Change (Return)'] = dataset.Close.pct_change()
dataset.head()

,Open,High,Low,Close,Volume,MA_10,MA_30,MA_50,MA_70,MA_90,...,EWM_50,EWM_70,EWM_90,RSI_14,RSI_24,RSI_34,RSI_44,RSI_54,RSI_64,Prct Change (Return)
Date,,,,,,,,,,,,,,,,,,,,,
2020-01-02,3244.67,3258.14,3235.53,3257.85,3458250000,NaN,NaN,NaN,NaN,NaN,...,3257.850000,3257.850000,3257.850000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-03,3226.36,3246.15,3222.34,3234.85,3461290000,NaN,NaN,NaN,NaN,NaN,...,3256.948039,3253.668182,3257.394554,NaN,NaN,NaN,NaN,NaN,NaN,-0.007060
2020-01-06,3217.55,3246.84,3214.64,3246.28,3674070000,NaN,NaN,NaN,NaN,NaN,...,3256.529685,3252.324876,3257.174464,NaN,NaN,NaN,NaN,NaN,NaN,0.003533
2020-01-07,3241.86,3244.91,3232.43,3237.18,3420380000,NaN,NaN,NaN,NaN,NaN,...,3255.770874,3249.571262,3256.778534,NaN,NaN,NaN,NaN,NaN,NaN,-0.002803
2020-01-08,3238.59,3267.07,3236.67,3253.05,3720890000,NaN,NaN,NaN,NaN,NaN,...,3255.664173,3250.203760,3256.704702,NaN,NaN,NaN,NaN,NaN,NaN,0.004902


# Next Day Percentage Change Prediction

In [13]:
dataset['Prct Change (Return)'] = dataset['Prct Change (Return)'].shift(-1)
dataset.head()

,Open,High,Low,Close,Volume,MA_10,MA_30,MA_50,MA_70,MA_90,...,EWM_50,EWM_70,EWM_90,RSI_14,RSI_24,RSI_34,RSI_44,RSI_54,RSI_64,Prct Change (Return)
Date,,,,,,,,,,,,,,,,,,,,,
2020-01-02,3244.67,3258.14,3235.53,3257.85,3458250000,NaN,NaN,NaN,NaN,NaN,...,3257.850000,3257.850000,3257.850000,NaN,NaN,NaN,NaN,NaN,NaN,-0.007060
2020-01-03,3226.36,3246.15,3222.34,3234.85,3461290000,NaN,NaN,NaN,NaN,NaN,...,3256.948039,3253.668182,3257.394554,NaN,NaN,NaN,NaN,NaN,NaN,0.003533
2020-01-06,3217.55,3246.84,3214.64,3246.28,3674070000,NaN,NaN,NaN,NaN,NaN,...,3256.529685,3252.324876,3257.174464,NaN,NaN,NaN,NaN,NaN,NaN,-0.002803
2020-01-07,3241.86,3244.91,3232.43,3237.18,3420380000,NaN,NaN,NaN,NaN,NaN,...,3255.770874,3249.571262,3256.778534,NaN,NaN,NaN,NaN,NaN,NaN,0.004902
2020-01-08,3238.59,3267.07,3236.67,3253.05,3720890000,NaN,NaN,NaN,NaN,NaN,...,3255.664173,3250.203760,3256.704702,NaN,NaN,NaN,NaN,NaN,NaN,0.006655


In [14]:
dataset.dropna(inplace=True)

In [15]:
dataset.shape

(414, 22)

## Training and Test set

In [16]:
# test set 
te = int(0.1 * len(dataset))

In [17]:
# Training and Test set
dataset_train = dataset.iloc[:-te,:]
dataset_test  = dataset.iloc[-te:,:]
dataset_train.shape, dataset_test.shape 

((373, 22), (41, 22))

In [18]:
# input and output
X_train, y_train = dataset_train.iloc[:,:-1], dataset_train.iloc[:,-1]
X_test, y_test   = dataset_test.iloc[:,:-1] , dataset_test.iloc[:,-1]

## XGBoost Model

In [19]:
from xgboost import XGBRegressor
xgb = XGBRegressor(random_state=0, objective ='reg:squarederror')
xgb.fit(X_train, y_train)

XGBRegressor(objective='reg:squarederror')

In [20]:
print('Training Score:' , xgb.score( X_train, y_train ).round(2) )
print('Test     Score:', xgb.score( X_test , y_test ).round(2)  )

Training Score: 0.62
Test     Score: -0.63


In [21]:
from sklearn.metrics import mean_squared_error as mse
train_error = mse( xgb.predict(X_train),y_train, squared=False)
test_error = mse( xgb.predict(X_test),y_test, squared=False)
train_error, test_error

(0.006142683782034494, 0.01230096930400372)

In [22]:
# graph of closing values
import plotly.graph_objects as go
from plotly.subplots import make_subplots
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x=X_train.index, y=xgb.predict(X_train), name='Prediction'), secondary_y=False)
fig.add_trace(go.Scatter(x=X_train.index, y=y_train, name='Actual'), secondary_y=False)
fig.update_layout(title_text='Prediction vs Actual')

# Action Item
- use some of the features as input and check the performance of the model


## Example

In [23]:
dataset.head()

,Open,High,Low,Close,Volume,MA_10,MA_30,MA_50,MA_70,MA_90,...,EWM_50,EWM_70,EWM_90,RSI_14,RSI_24,RSI_34,RSI_44,RSI_54,RSI_64,Prct Change (Return)
Date,,,,,,,,,,,,,,,,,,,,,
2020-05-11,2915.46,2944.25,2903.44,2930.19,4807320000,2884.682,2778.140333,2729.7126,2885.430286,2973.298444,...,2839.873385,2880.007855,2939.323555,69.983694,63.017379,67.182718,52.574799,52.574799,52.574799,-0.020500
2020-05-12,2939.50,2945.82,2869.59,2870.12,5107710000,2885.355,2786.256000,2725.3104,2880.353143,2968.990333,...,2841.059526,2878.210063,2937.953187,57.348637,59.759882,61.328232,49.826520,49.826520,49.826520,-0.017463
2020-05-13,2865.86,2874.14,2793.15,2820.00,6143130000,2873.404,2794.103000,2721.6430,2874.225714,2964.380889,...,2840.233663,2867.626416,2935.617481,52.092760,53.369521,59.121919,51.156319,51.156319,51.156319,0.011525
2020-05-14,2794.54,2852.80,2766.64,2852.50,5641920000,2867.411,2806.836333,2716.0906,2867.867286,2960.005556,...,2840.714695,2864.876158,2933.971590,51.503932,53.037764,56.297672,55.862821,55.862821,55.862821,0.003926
2020-05-15,2829.95,2865.01,2816.78,2863.70,5477040000,2870.710,2818.063000,2712.8858,2861.138857,2955.855778,...,2841.616080,2864.662311,2932.580073,48.502290,55.029615,59.541506,52.585903,52.585903,52.585903,0.031501


In [24]:
dataset.shape [1]

22

In [25]:
import pandas as pd

## Only one feature

In [26]:
te = int(0.1 * len(dataset))
df_result = pd.DataFrame(columns=['Feature', 'Training Score', 'Test Score'])
for i in range(dataset.shape[1]-1):
    dataset_train = dataset.iloc[:-te,i]
    dataset_test  = dataset.iloc[-te:,i]
    X_train, y_train = dataset_train.values.reshape(dataset_train.shape[0],1), dataset_train.values
    X_test, y_test   = dataset_test.values.reshape(dataset_test.shape[0],1) , dataset_test.values
    xgb = XGBRegressor(random_state=0,objective ='reg:squarederror')
    xgb.fit(X_train, y_train)
    # print(dataset.columns[i])
    # print('Training RMSE Score:' , xgb.score( X_train, y_train ).round(2) )
    # print('Test     RMSE  Score:', xgb.score( X_test , y_test ).round(2)  )
    df_result.loc[len(df_result)] = [dataset.columns[i], xgb.score( X_train, y_train ), xgb.score( X_test , y_test )]

In [27]:
df_result.round(2).sort_values('Test Score', ascending=False)

,Feature,Training Score,Test Score
20,RSI_64,1.0,1.00
15,RSI_14,1.0,1.00
4,Volume,1.0,1.00
19,RSI_54,1.0,1.00
18,RSI_44,1.0,1.00
17,RSI_34,1.0,1.00
16,RSI_24,1.0,1.00
3,Close,1.0,-1.20
2,Low,1.0,-1.29
0,Open,1.0,-2.51


In [28]:
dataset_test.shape

(41,)

## All features

In [29]:
te = int(0.1 * len(dataset))

dataset_train = dataset[['Volume', 'RSI_14', 'RSI_24', 'Prct Change (Return)']].iloc[:-te,:]
dataset_test  = dataset[['Volume', 'RSI_14', 'RSI_24', 'Prct Change (Return)']].iloc[-te:,:]

X_train, y_train = dataset_train.iloc[:,:-1], dataset_train.iloc[:,-1]
X_test, y_test   = dataset_test.iloc[:,:-1] , dataset_test.iloc[:,-1]

xgb = XGBRegressor(random_state=0, objective ='reg:squarederror')
xgb.fit(X_train, y_train)


print('Training Score:' , xgb.score( X_train, y_train ).round(2) )
print('Test     Score:', xgb.score( X_test , y_test ).round(2)  )


Training Score: 0.48
Test     Score: -0.77


## Two features

In [30]:
from itertools import combinations
combinations(df_result.columns, 2)

In [31]:
list(combinations(dataset.columns[:-1], 2))

[('Open', 'High'),
 ('Open', 'Low'),
 ('Open', 'Close'),
 ('Open', 'Volume'),
 ('Open', 'MA_10'),
 ('Open', 'MA_30'),
 ('Open', 'MA_50'),
 ('Open', 'MA_70'),
 ('Open', 'MA_90'),
 ('Open', 'EWM_10'),
 ('Open', 'EWM_30'),
 ('Open', 'EWM_50'),
 ('Open', 'EWM_70'),
 ('Open', 'EWM_90'),
 ('Open', 'RSI_14'),
 ('Open', 'RSI_24'),
 ('Open', 'RSI_34'),
 ('Open', 'RSI_44'),
 ('Open', 'RSI_54'),
 ('Open', 'RSI_64'),
 ('High', 'Low'),
 ('High', 'Close'),
 ('High', 'Volume'),
 ('High', 'MA_10'),
 ('High', 'MA_30'),
 ('High', 'MA_50'),
 ('High', 'MA_70'),
 ('High', 'MA_90'),
 ('High', 'EWM_10'),
 ('High', 'EWM_30'),
 ('High', 'EWM_50'),
 ('High', 'EWM_70'),
 ('High', 'EWM_90'),
 ('High', 'RSI_14'),
 ('High', 'RSI_24'),
 ('High', 'RSI_34'),
 ('High', 'RSI_44'),
 ('High', 'RSI_54'),
 ('High', 'RSI_64'),
 ('Low', 'Close'),
 ('Low', 'Volume'),
 ('Low', 'MA_10'),
 ('Low', 'MA_30'),
 ('Low', 'MA_50'),
 ('Low', 'MA_70'),
 ('Low', 'MA_90'),
 ('Low', 'EWM_10'),
 ('Low', 'EWM_30'),
 ('Low', 'EWM_50'),
 ('Low'

In [32]:
dataset.head()

,Open,High,Low,Close,Volume,MA_10,MA_30,MA_50,MA_70,MA_90,...,EWM_50,EWM_70,EWM_90,RSI_14,RSI_24,RSI_34,RSI_44,RSI_54,RSI_64,Prct Change (Return)
Date,,,,,,,,,,,,,,,,,,,,,
2020-05-11,2915.46,2944.25,2903.44,2930.19,4807320000,2884.682,2778.140333,2729.7126,2885.430286,2973.298444,...,2839.873385,2880.007855,2939.323555,69.983694,63.017379,67.182718,52.574799,52.574799,52.574799,-0.020500
2020-05-12,2939.50,2945.82,2869.59,2870.12,5107710000,2885.355,2786.256000,2725.3104,2880.353143,2968.990333,...,2841.059526,2878.210063,2937.953187,57.348637,59.759882,61.328232,49.826520,49.826520,49.826520,-0.017463
2020-05-13,2865.86,2874.14,2793.15,2820.00,6143130000,2873.404,2794.103000,2721.6430,2874.225714,2964.380889,...,2840.233663,2867.626416,2935.617481,52.092760,53.369521,59.121919,51.156319,51.156319,51.156319,0.011525
2020-05-14,2794.54,2852.80,2766.64,2852.50,5641920000,2867.411,2806.836333,2716.0906,2867.867286,2960.005556,...,2840.714695,2864.876158,2933.971590,51.503932,53.037764,56.297672,55.862821,55.862821,55.862821,0.003926
2020-05-15,2829.95,2865.01,2816.78,2863.70,5477040000,2870.710,2818.063000,2712.8858,2861.138857,2955.855778,...,2841.616080,2864.662311,2932.580073,48.502290,55.029615,59.541506,52.585903,52.585903,52.585903,0.031501


In [33]:
te = int(0.1 * len(dataset))

df_result2 = pd.DataFrame(columns=['Feature', 'Training Error', 'Test Error'])

for i in combinations(dataset.columns[:-1], 2):

    dataset_train = dataset[list(i) + ['Prct Change (Return)']].iloc[:-te,:]
    dataset_test  = dataset[list(i) + ['Prct Change (Return)']].iloc[-te:,:]

    X_train, y_train = dataset_train.iloc[:,:-1], dataset_train.iloc[:,-1]
    X_test, y_test   = dataset_test.iloc[:,:-1] , dataset_test.iloc[:,-1]

    xgb = XGBRegressor(random_state=0, objective ='reg:squarederror')
    xgb.fit(X_train, y_train)


    train_error = mse( xgb.predict(X_train),y_train, squared=False)
    test_error = mse( xgb.predict(X_test),y_test, squared=False)


    df_result2.loc[len(df_result2)] = [i,train_error, test_error]

In [34]:
df_result2.sort_values('Test Error').head()

,Feature,Training Error,Test Error
48,"(Low, EWM_50)",0.007630,0.009315
41,"(Low, MA_10)",0.007601,0.009361
79,"(Volume, EWM_10)",0.007499,0.009366
80,"(Volume, EWM_30)",0.007499,0.009366
82,"(Volume, EWM_70)",0.007499,0.009366


## Three features

In [35]:
from itertools import combinations
combinations(df_result.columns, 3)

In [36]:
list(combinations(dataset.columns[:-1], 3))

[('Open', 'High', 'Low'),
 ('Open', 'High', 'Close'),
 ('Open', 'High', 'Volume'),
 ('Open', 'High', 'MA_10'),
 ('Open', 'High', 'MA_30'),
 ('Open', 'High', 'MA_50'),
 ('Open', 'High', 'MA_70'),
 ('Open', 'High', 'MA_90'),
 ('Open', 'High', 'EWM_10'),
 ('Open', 'High', 'EWM_30'),
 ('Open', 'High', 'EWM_50'),
 ('Open', 'High', 'EWM_70'),
 ('Open', 'High', 'EWM_90'),
 ('Open', 'High', 'RSI_14'),
 ('Open', 'High', 'RSI_24'),
 ('Open', 'High', 'RSI_34'),
 ('Open', 'High', 'RSI_44'),
 ('Open', 'High', 'RSI_54'),
 ('Open', 'High', 'RSI_64'),
 ('Open', 'Low', 'Close'),
 ('Open', 'Low', 'Volume'),
 ('Open', 'Low', 'MA_10'),
 ('Open', 'Low', 'MA_30'),
 ('Open', 'Low', 'MA_50'),
 ('Open', 'Low', 'MA_70'),
 ('Open', 'Low', 'MA_90'),
 ('Open', 'Low', 'EWM_10'),
 ('Open', 'Low', 'EWM_30'),
 ('Open', 'Low', 'EWM_50'),
 ('Open', 'Low', 'EWM_70'),
 ('Open', 'Low', 'EWM_90'),
 ('Open', 'Low', 'RSI_14'),
 ('Open', 'Low', 'RSI_24'),
 ('Open', 'Low', 'RSI_34'),
 ('Open', 'Low', 'RSI_44'),
 ('Open', 'Low',

In [37]:
te = int(0.1 * len(dataset))

df_result3 = pd.DataFrame(columns=['Feature', 'Training Error', 'Test Error'])

for i in combinations(dataset.columns[:-1], 3):

    dataset_train = dataset[list(i) + ['Prct Change (Return)']].iloc[:-te,:]
    dataset_test  = dataset[list(i) + ['Prct Change (Return)']].iloc[-te:,:]

    X_train, y_train = dataset_train.iloc[:,:-1], dataset_train.iloc[:,-1]
    X_test, y_test   = dataset_test.iloc[:,:-1] , dataset_test.iloc[:,-1]

    xgb = XGBRegressor(random_state=0, objective ='reg:squarederror')
    xgb.fit(X_train, y_train)


    train_error = mse( xgb.predict(X_train),y_train, squared=False)
    test_error = mse( xgb.predict(X_test),y_test, squared=False)


    df_result3.loc[len(df_result3)] = [i,train_error, test_error]

In [38]:
df_result3.sort_values('Test Error').head()

,Feature,Training Error,Test Error
702,"(Volume, MA_70, RSI_54)",0.006891,0.009191
703,"(Volume, MA_70, RSI_64)",0.006891,0.009191
701,"(Volume, MA_70, RSI_44)",0.006891,0.009191
378,"(Low, Volume, MA_10)",0.007426,0.009295
400,"(Low, MA_10, EWM_50)",0.007328,0.009319


# Decision Tree

In [39]:
from sklearn.tree import DecisionTreeRegressor

te = int(0.1 * len(dataset))

df_result_dt = pd.DataFrame(columns=['Feature', 'Training Error', 'Test Error'])

for i in combinations(dataset.columns[:-1], 3):

    dataset_train = dataset[list(i) + ['Prct Change (Return)']].iloc[:-te,:]
    dataset_test  = dataset[list(i) + ['Prct Change (Return)']].iloc[-te:,:]

    X_train, y_train = dataset_train.iloc[:,:-1], dataset_train.iloc[:,-1]
    X_test, y_test   = dataset_test.iloc[:,:-1] , dataset_test.iloc[:,-1]

    dt = DecisionTreeRegressor(random_state=0)
    dt.fit(X_train, y_train)


    train_error = mse( dt.predict(X_train),y_train, squared=False)
    test_error = mse( dt.predict(X_test),y_test, squared=False)


    df_result_dt.loc[len(df_result_dt)] = [i,train_error, test_error]

In [40]:
df_result_dt.sort_values('Test Error').head()

,Feature,Training Error,Test Error
380,"(Low, Volume, MA_50)",0.0,0.008697
514,"(Close, Volume, MA_10)",0.0,0.008703
519,"(Close, Volume, EWM_10)",0.0,0.008703
520,"(Close, Volume, EWM_30)",0.0,0.008703
522,"(Close, Volume, EWM_70)",0.0,0.008703


In [41]:
#max depth=3
from sklearn.tree import DecisionTreeRegressor

te = int(0.1 * len(dataset))

df_result_dt = pd.DataFrame(columns=['Feature', 'Training Error', 'Test Error'])

for i in combinations(dataset.columns[:-1], 3):

    dataset_train = dataset[list(i) + ['Prct Change (Return)']].iloc[:-te,:]
    dataset_test  = dataset[list(i) + ['Prct Change (Return)']].iloc[-te:,:]

    X_train, y_train = dataset_train.iloc[:,:-1], dataset_train.iloc[:,-1]
    X_test, y_test   = dataset_test.iloc[:,:-1] , dataset_test.iloc[:,-1]

    dt = DecisionTreeRegressor(max_depth=3)
    dt.fit(X_train, y_train)


    train_error = mse( dt.predict(X_train),y_train, squared=False)
    test_error = mse( dt.predict(X_test),y_test, squared=False)


    df_result_dt.loc[len(df_result_dt)] = [i,train_error, test_error]

In [42]:
df_result_dt.sort_values('Test Error').head()

,Feature,Training Error,Test Error
664,"(Volume, MA_10, RSI_64)",0.008815,0.009222
703,"(Volume, MA_70, RSI_64)",0.008815,0.009222
702,"(Volume, MA_70, RSI_54)",0.008815,0.009222
701,"(Volume, MA_70, RSI_44)",0.008815,0.009222
663,"(Volume, MA_10, RSI_54)",0.008815,0.009222


In [43]:
#max depth=4
from sklearn.tree import DecisionTreeRegressor

te = int(0.1 * len(dataset))

df_result_dt = pd.DataFrame(columns=['Feature', 'Training Error', 'Test Error'])

for i in combinations(dataset.columns[:-1], 3):

    dataset_train = dataset[list(i) + ['Prct Change (Return)']].iloc[:-te,:]
    dataset_test  = dataset[list(i) + ['Prct Change (Return)']].iloc[-te:,:]

    X_train, y_train = dataset_train.iloc[:,:-1], dataset_train.iloc[:,-1]
    X_test, y_test   = dataset_test.iloc[:,:-1] , dataset_test.iloc[:,-1]

    dt = DecisionTreeRegressor(max_depth=4)
    dt.fit(X_train, y_train)


    train_error = mse( dt.predict(X_train),y_train, squared=False)
    test_error = mse( dt.predict(X_test),y_test, squared=False)


    df_result_dt.loc[len(df_result_dt)] = [i,train_error, test_error]

In [44]:
df_result_dt.sort_values('Test Error').head()

,Feature,Training Error,Test Error
712,"(Volume, MA_90, RSI_44)",0.008143,0.009179
713,"(Volume, MA_90, RSI_54)",0.008143,0.009179
714,"(Volume, MA_90, RSI_64)",0.008143,0.009179
664,"(Volume, MA_10, RSI_64)",0.008449,0.009182
529,"(Close, Volume, RSI_64)",0.008439,0.009182


# Knn Regressor

In [45]:
# neighbors=3
from sklearn.neighbors import KNeighborsRegressor

te = int(0.1 * len(dataset))

df_result_knn = pd.DataFrame(columns=['Feature', 'Training Error', 'Test Error'])

for i in combinations(dataset.columns[:-1], 3):

    dataset_train = dataset[list(i) + ['Prct Change (Return)']].iloc[:-te,:]
    dataset_test  = dataset[list(i) + ['Prct Change (Return)']].iloc[-te:,:]

    X_train, y_train = dataset_train.iloc[:,:-1], dataset_train.iloc[:,-1]
    X_test, y_test   = dataset_test.iloc[:,:-1] , dataset_test.iloc[:,-1]

    knn = KNeighborsRegressor(n_neighbors=3)
    knn.fit(X_train, y_train)


    train_error = mse( knn.predict(X_train),y_train, squared=False)
    test_error = mse( knn.predict(X_test),y_test, squared=False)


    df_result_knn.loc[len(df_result_knn)] = [i,train_error, test_error]

In [46]:
df_result_knn.sort_values('Test Error').head()

,Feature,Training Error,Test Error
327,"(High, EWM_50, RSI_14)",0.008190,0.009020
312,"(High, EWM_10, RSI_34)",0.008114,0.009034
321,"(High, EWM_30, RSI_34)",0.008114,0.009034
336,"(High, EWM_70, RSI_34)",0.008114,0.009034
441,"(Low, MA_70, EWM_90)",0.007815,0.009051


In [47]:
# neighbors=4
from sklearn.neighbors import KNeighborsRegressor

te = int(0.1 * len(dataset))

df_result_knn = pd.DataFrame(columns=['Feature', 'Training Error', 'Test Error'])

for i in combinations(dataset.columns[:-1], 3):

    dataset_train = dataset[list(i) + ['Prct Change (Return)']].iloc[:-te,:]
    dataset_test  = dataset[list(i) + ['Prct Change (Return)']].iloc[-te:,:]

    X_train, y_train = dataset_train.iloc[:,:-1], dataset_train.iloc[:,-1]
    X_test, y_test   = dataset_test.iloc[:,:-1] , dataset_test.iloc[:,-1]

    knn = KNeighborsRegressor(n_neighbors=4)
    knn.fit(X_train, y_train)


    train_error = mse( knn.predict(X_train),y_train, squared=False)
    test_error = mse( knn.predict(X_test),y_test, squared=False)


    df_result_knn.loc[len(df_result_knn)] = [i,train_error, test_error]

In [48]:
df_result_knn.sort_values('Test Error').head()

,Feature,Training Error,Test Error
1322,"(RSI_24, RSI_34, RSI_64)",0.008113,0.008886
1321,"(RSI_24, RSI_34, RSI_54)",0.008113,0.008886
1320,"(RSI_24, RSI_34, RSI_44)",0.008113,0.008886
566,"(Close, MA_50, RSI_14)",0.008497,0.009260
629,"(Close, EWM_90, RSI_14)",0.008570,0.009283


# Linear Regressor

In [49]:
from sklearn.linear_model import LinearRegression

te = int(0.1 * len(dataset))

df_result_lin_reg = pd.DataFrame(columns=['Feature', 'Training Error', 'Test Error'])

for i in combinations(dataset.columns[:-1], 3):

    dataset_train = dataset[list(i) + ['Prct Change (Return)']].iloc[:-te,:]
    dataset_test  = dataset[list(i) + ['Prct Change (Return)']].iloc[-te:,:]

    X_train, y_train = dataset_train.iloc[:,:-1], dataset_train.iloc[:,-1]
    X_test, y_test   = dataset_test.iloc[:,:-1] , dataset_test.iloc[:,-1]

    lin_reg = LinearRegression()
    lin_reg.fit(X_train, y_train)


    train_error = mse( lin_reg.predict(X_train),y_train, squared=False)
    test_error = mse( lin_reg.predict(X_test),y_test, squared=False)


    df_result_lin_reg.loc[len(df_result_lin_reg)] = [i,train_error, test_error]

In [50]:
df_result_lin_reg.sort_values('Test Error').head()

,Feature,Training Error,Test Error
631,"(Close, EWM_90, RSI_34)",0.009753,0.009169
541,"(Close, MA_10, RSI_34)",0.009776,0.009169
850,"(MA_10, EWM_70, RSI_34)",0.009798,0.009173
835,"(MA_10, EWM_30, RSI_34)",0.009798,0.009173
826,"(MA_10, EWM_10, RSI_34)",0.009798,0.009173


# Random Forest Regressor

In [51]:
from sklearn.ensemble import RandomForestRegressor

te = int(0.1 * len(dataset))

df_result_rf = pd.DataFrame(columns=['Feature', 'Training Error', 'Test Error'])

for i in combinations(dataset.columns[:-1], 3):

    dataset_train = dataset[list(i) + ['Prct Change (Return)']].iloc[:-te,:]
    dataset_test  = dataset[list(i) + ['Prct Change (Return)']].iloc[-te:,:]

    X_train, y_train = dataset_train.iloc[:,:-1], dataset_train.iloc[:,-1]
    X_test, y_test   = dataset_test.iloc[:,:-1] , dataset_test.iloc[:,-1]

    rf = RandomForestRegressor(random_state=0)
    rf.fit(X_train, y_train)


    train_error = mse( rf.predict(X_train),y_train, squared=False)
    test_error = mse( rf.predict(X_test),y_test, squared=False)


    df_result_rf.loc[len(df_result_rf)] = [i,train_error, test_error]

In [52]:
df_result_rf.sort_values('Test Error').head()

,Feature,Training Error,Test Error
1325,"(RSI_24, RSI_54, RSI_64)",0.003881,0.008509
1324,"(RSI_24, RSI_44, RSI_64)",0.003881,0.008509
1323,"(RSI_24, RSI_44, RSI_54)",0.003881,0.008509
1321,"(RSI_24, RSI_34, RSI_54)",0.003734,0.008624
1322,"(RSI_24, RSI_34, RSI_64)",0.003734,0.008624


In [53]:
from sklearn.ensemble import RandomForestRegressor

te = int(0.1 * len(dataset))

df_result_rf = pd.DataFrame(columns=['Feature', 'Training Error', 'Test Error'])

for i in combinations(dataset.columns[:-1], 3):

    dataset_train = dataset[list(i) + ['Prct Change (Return)']].iloc[:-te,:]
    dataset_test  = dataset[list(i) + ['Prct Change (Return)']].iloc[-te:,:]

    X_train, y_train = dataset_train.iloc[:,:-1], dataset_train.iloc[:,-1]
    X_test, y_test   = dataset_test.iloc[:,:-1] , dataset_test.iloc[:,-1]

    rf = RandomForestRegressor(n_estimators=3, max_depth=2)
    rf.fit(X_train, y_train)


    train_error = mse( rf.predict(X_train),y_train, squared=False)
    test_error = mse( rf.predict(X_test),y_test, squared=False)


    df_result_rf.loc[len(df_result_rf)] = [i,train_error, test_error]

In [54]:
df_result_rf.sort_values('Test Error').head()

,Feature,Training Error,Test Error
1279,"(EWM_70, EWM_90, RSI_64)",0.009551,0.009150
111,"(Open, MA_50, RSI_64)",0.009536,0.009174
467,"(Low, EWM_10, RSI_54)",0.009610,0.009198
1159,"(MA_90, RSI_34, RSI_44)",0.009527,0.009229
1207,"(EWM_10, RSI_44, RSI_54)",0.009503,0.009245


# Action Item: 6/24/2022
- Use different ML algorithms
  - linear regression
  - knnRegressor
  - Decision Tree Regressor
  - Random FOrest Regressor
- try different hyperparameters
  - DT, RF, XGB: max_depth=2,3,4
 

# Action Item 4/1/2022
- try different time intervals
- try increasning number of features like MA_20, Ma_30,.....
- try combinations of 4 features

# Action Item 3/18/2022
- Try two columns at a time

##Four Features

In [55]:
from itertools import combinations
combinations(df_result.columns, 4)

In [56]:
list(combinations(dataset.columns[:-1], 4))

[('Open', 'High', 'Low', 'Close'),
 ('Open', 'High', 'Low', 'Volume'),
 ('Open', 'High', 'Low', 'MA_10'),
 ('Open', 'High', 'Low', 'MA_30'),
 ('Open', 'High', 'Low', 'MA_50'),
 ('Open', 'High', 'Low', 'MA_70'),
 ('Open', 'High', 'Low', 'MA_90'),
 ('Open', 'High', 'Low', 'EWM_10'),
 ('Open', 'High', 'Low', 'EWM_30'),
 ('Open', 'High', 'Low', 'EWM_50'),
 ('Open', 'High', 'Low', 'EWM_70'),
 ('Open', 'High', 'Low', 'EWM_90'),
 ('Open', 'High', 'Low', 'RSI_14'),
 ('Open', 'High', 'Low', 'RSI_24'),
 ('Open', 'High', 'Low', 'RSI_34'),
 ('Open', 'High', 'Low', 'RSI_44'),
 ('Open', 'High', 'Low', 'RSI_54'),
 ('Open', 'High', 'Low', 'RSI_64'),
 ('Open', 'High', 'Close', 'Volume'),
 ('Open', 'High', 'Close', 'MA_10'),
 ('Open', 'High', 'Close', 'MA_30'),
 ('Open', 'High', 'Close', 'MA_50'),
 ('Open', 'High', 'Close', 'MA_70'),
 ('Open', 'High', 'Close', 'MA_90'),
 ('Open', 'High', 'Close', 'EWM_10'),
 ('Open', 'High', 'Close', 'EWM_30'),
 ('Open', 'High', 'Close', 'EWM_50'),
 ('Open', 'High', 'Cl

In [57]:
te = int(0.1 * len(dataset))

df_result4 = pd.DataFrame(columns=['Feature', 'Training Error', 'Test Error'])

for i in combinations(dataset.columns[:-1], 4):

    dataset_train = dataset[list(i) + ['Prct Change (Return)']].iloc[:-te,:]
    dataset_test  = dataset[list(i) + ['Prct Change (Return)']].iloc[-te:,:]

    X_train, y_train = dataset_train.iloc[:,:-1], dataset_train.iloc[:,-1]
    X_test, y_test   = dataset_test.iloc[:,:-1] , dataset_test.iloc[:,-1]

    xgb = XGBRegressor(random_state=0, objective ='reg:squarederror')
    xgb.fit(X_train, y_train)


    train_error = mse( xgb.predict(X_train),y_train, squared=False)
    test_error = mse( xgb.predict(X_test),y_test, squared=False)


    df_result4.loc[len(df_result4)] = [i,train_error, test_error]

In [58]:
df_result4.sort_values('Test Error').head()

,Feature,Training Error,Test Error
3943,"(Volume, MA_70, RSI_44, RSI_64)",0.006891,0.009191
3942,"(Volume, MA_70, RSI_44, RSI_54)",0.006891,0.009191
3944,"(Volume, MA_70, RSI_54, RSI_64)",0.006891,0.009191
2383,"(Low, MA_10, MA_50, EWM_50)",0.007298,0.009258
295,"(Open, Low, EWM_50, RSI_64)",0.007043,0.009260
